<a href="https://colab.research.google.com/github/MarigoldJ/ygl2/blob/main/class/20210618_nlp_day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Today's Topic

* RNN의 문제점
    * 기울기 소실
    * 번역에는 사용하기 힘들다?
        * 각 나라별 어순 정보 파악 힘들다.

# Seq2seq 구현하기

## LSTM Encoder

In [1]:
import tensorflow as tf

In [23]:
class Encoder(tf.keras.Model):
    '''
    seq2seq의 encoder
    '''
    def __init__(self, vocab_size, embedding_dim, encoder_units):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(encoder_units)
        # return_sequences 매개변수를 기본값 False로 전달
    
    def call(self, x):      # __call__ 과 비슷한 듯
        '''
        x를 넣고 중간에 데이터의 shape을 디버깅
        '''
        print('입력할 때 shape :', x.shape)

        e = self.embedding(x)
        print("Embedding Layer를 거친 뒤 shape :", e.shape)

        output_v = self.lstm(e)
        print("LSTM Layer를 거친 뒤 shape :", output_v.shape)

        return output_v


In [24]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print('Vocab Size :', vocab_size)
print('Embedding Size :', emb_size)
print('LSTM Size :', lstm_size)
print('Batch Size :', batch_size)
print('Sample Sequence Length :', sample_seq_len)

Vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length : 3


In [25]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_encoder_input = tf.zeros((batch_size, sample_seq_len))

sample_encoder_output = encoder(sample_input)
# 인코더 LSTM의 최종 State (이후 컨벡스트 벡터로 사용될 예정)

입력할 때 shape : (1, 3)
Embedding Layer를 거친 뒤 shape : (1, 3, 256)
LSTM Layer를 거친 뒤 shape : (1, 512)


![](https://aiffelstaticprd.blob.core.windows.net/media/images/GN-4-L-6.max-800x600.jpg)

## LSTM Decoder

In [29]:
class Decoder(tf.keras.Model):
    '''
    seq2seq의 decoder
    '''
    def __init__(self, vocab_size, embedding_dim, decoder_units):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(decoder_units, return_sequences=True)
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, x, context_v):
        '''
        디코더의 입력 x와 인코더의 컨벡스트 벡터(final state)를 인자로 받음
        중간의 데이터들의 shape을 디버깅
        '''
        print('입력할 때 shape :', x.shape)

        e = self.embedding(x)
        print("Embedding Layer를 거친 뒤 shape :", e.shape)

        # 컨벡스트 벡터의 브로드캐스팅, 확장
        context_vh = tf.repeat(tf.expand_dims(context_v, axis=1), repeats=x.shape[1], axis=1)
        eh = tf.concat([e, context_vh], axis=-1)
        print('Context Vector가 합쳐진 shape :', eh.shape)

        output_v = self.lstm(eh)
        print("LSTM Layer를 거친 뒤 shape :", output_v.shape)

        output = self.fc(output_v)
        print('Decoder 최종 output의 shape :', output.shape)

        return output

In [30]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print('Vocab Size :', vocab_size)
print('Embedding Size :', emb_size)
print('LSTM Size :', lstm_size)
print('Batch Size :', batch_size)
print('Sample Sequence Length :', sample_seq_len)

Vocab Size : 30000
Embedding Size : 256
LSTM Size : 512
Batch Size : 1
Sample Sequence Length : 3


In [31]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_decoder_input = tf.zeros((batch_size, sample_seq_len))

sample_decoder_output = decoder(sample_decoder_input, sample_encoder_output)
# Decoder.call(x, context_v)를 호출


입력할 때 shape : (1, 3)
Embedding Layer를 거친 뒤 shape : (1, 3, 256)
Context Vector가 합쳐진 shape : (1, 3, 768)
LSTM Layer를 거친 뒤 shape : (1, 3, 512)
Decoder 최종 output의 shape : (1, 3, 30000)


# 어텐션 메커니즘 (Attention Mechanism)

* 기존의 RNN에 기반한 seq2seq 모델의 문제점
    1. 기억 소실, 기울기 소실
    2. 하나의 고정된 벡터에 모든 정보를 압축하려다 보니 정보 손실이 발생
* 어텐션 아이디어
    * 디코더에서 출력단어를 예측하는 매 시점마다, 인코더에서의 *전체* 입력 문장을 다시 한번 참고함
    * 전체 입력 문장을 모두 동일한 비율로 참고하는 것이 아니라, 해당 시점에 예측할 단어와 연관성 있는 단어 부분을 중점적으로 참고함

In [32]:
dictionary = {'2017': 'Transformer', '2018': 'BERT'}

print(dictionary['2017'])
print(dictionary['2018'])

Transformer
BERT


![](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)
* Query : t 시점의 디코더 셀에서의 hidden state
* Key : 모든 시점의 인코더 셀에서의 hidden state
* Value : 모든 시점의 인코더 셀의 hidden state

# 닷 프로덕트 어텐션 (Dot-Product Attention)

## 이론 설명

![](https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG)

그림 설명
* 디코더에서 sos, je를 넣어 je, suis를 예측한 뒤, 세번째 단어를 예측하려는 시점
* suis 다음 단어를 예측하기 위해서, 인코더의 hidden state들을 참조하여 다음 단어 etudiant를 예측해내는 모습

### 1.Attention Score 구하기

![](https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG)


그림 설명
* 디코더의 현재 시점 t에서의 state와, 인코더의 모든 시점에서의 state의 유사도를 점수화한다.(내적을 통해 유사도를 계산하는 듯)
* 수식은 아래와 같다.
$$ score(s_t, h_i) = s_t^T h_i $$
$$ e^t = [ score(s_t, h_1), score(s_t, h_2), ..., score(s_t, h_N) ] $$
    * $s_t$는 디코더의 현재 시점 t에서의 state (행벡터)
    * $h_i$는 인코더의 모든 시점 중 특정 시점 i에서의 state (행벡터)
    * $e^t$는 디코더의 현재 시점 t에서의 attention score 모음값


### 2.Attention Distribution 구하기

* 소프트맥스(softmax) 함수를 통해 어텐션 분포를 구한다

![](https://wikidocs.net/images/page/22893/dotproductattention3_final.PNG)

그림 설명
* 1에서 계산한 각 시점별 Attention Score를 softmax 함수로 처리한다.
* 수식은 아래와 같다.
$$ a^t = softmax(e^t) $$
    * $a^t$는 어텐션 가중치 모음값인 attention distribution
    * $e^t$는 디코더의 현재 시점 t에서의 attention score 모음값

### 3.Attention Value 구하기

* 각 인코더의 어텐션 가중치와 은닉 상태를 가중합해서 어텐션 값(attention value)을 구한다.

![](https://wikidocs.net/images/page/22893/dotproductattention4_final.PNG)

그림 설명
* 2에서 계산한 각 시점별 가중치인 Attention distribution과, 각 시점별 state를 곱하여 Attention value를 구한다.
    * 이 결과 값은 종합적인 정보를 담은 state라고 생각할 수 있다.
    * 인코더에서 디코더로 전달하는 state를 만든 것이다.
        * 기존 : 인코더의 마지막 state
        * 현재 하는 것 : 인코더의 state를 종합적으로 고려한 새로운 state
* 수식은 아래와 같다.
$$ a_t = \sum^{N}_{i=1}{a^t_i h_i} $$
    * $a^t$는 어텐션 가중치 모음값인 attention distribution
        * 해당 시점을 얼마나 반영할지 나타낸다고 생각하기
    * $h_i$는 인코더의 특정 시점 i에서의 state (행벡터)
    * $a_t$는 가중치가 반영된 state (행벡터)
        * $h_i$ 들을 대표하는 state라고 생각하면 좋을 듯

### 4.Attention value, decoder state 연결하기

* 3에서 구한 어텐션 값과, 기존의 디코더의 현재 시점 t의 state를 연결한다.
* 새로운 벡터를 만드는 것이다.

![](https://wikidocs.net/images/page/22893/dotproductattention5_final_final.PNG)

그림 설명
* 3에서 계산한 인코더의 종합 State인 Attention Value와, 기존의 디코더의 현재 시점 t의 state를 연결하여 새로운 벡터를 만든다.
* 굳이 수식으로 쓰면 아래와 같다.
$$ v_t = [a_t;s_t] $$
    * $v_t$는 $a_t$와 $s_t$를 단순히 연결한 것. (행벡터)
    * $a_t$는 가중치가 반영된 인코더의 state (행벡터)
        * 인코더의 정보를 종합한 state로 보면 좋을 듯!
    * $s_t$는 디코더의 현재 시점 t에서의 state (행벡터)

### 5.출력층 연산의 입력 벡터 만들기

* 논문 내용에 따르면, 4에서의 $v_t$가 출력층으로 가기전에 한가지 연산을 거침.
* 연산이라 함은 $tanh$ 함수를 거치는 연산을 의미함.

![](https://wikidocs.net/images/page/22893/st.PNG)

그림 설명
* 어텐션 메커니즘을 사용하기 전에는, 출력층의 입력으로 $s_t$(디코더에서 시점 t의 state) 가 사용됨.
* 어텐션 메커니즘을 사용하는 지금은, 출력층의 입력으로 $\tilde{s_t}$(attention이 반영된 디코더에서 시점 t의 state) 가 사용됨.
* $\tilde{s_t}$를 구하는 수식은 아래와 같다.
$$ \tilde{s_t}=tanh(W_c v_t + b_c) $$
    * $v_t$는 $a_t$와 $s_t$를 단순히 연결한 것. (행벡터)
    * $W_c$는 학습 가능한 가중치 행렬
    * $b_c$는 편향
    * $\tilde{s_t}$는 attention이 반영된 새로운 $s_t$(디코더의 현재 시점 t에서의 state) (행벡터)

### 6.출력층으로부터 예측 벡터 얻기

* 5에서 구한 출력층의 입력벡터를 출력층에 넣고 연산하여, 예측 벡터($\tilde{y_t}$)를 얻는다.

* 수식은 아래와 같다. 
$$ \tilde{y_t}=softmax(W_y \tilde{s_t} + b_y) $$
    * $\tilde{s_t}$는 attention이 반영된 새로운 $s_t$(디코더의 현재 시점 t에서의 state) (행벡터)
    * $W_y$는 학습 가능한 가중치 행렬
    * $b_y$는 편향
    * $\tilde{y_t}$는 예측벡터 (이 값을 문자로 바꾸면 예측 문자가 된다)


# Bahdanau Attention 구현

* attention score를 얻는 식
$$ score_{alignment} = W \bullet tanh(W_{decoder} H_{decoder} + W_{encoder} H_{encoder}) $$

In [ ]:
class BahdanauAttention(tf.keras.layer.Layer):    

    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_decoder = tf.keras.layers.Dense(units)
        self.w_encoder = tf.keras.layers.Dense(units)
        self.w_combine = tf.keras.layers.Dense(1)
    
    def call(self, h_encoder, h_decoder):
        '''
        h_encoder : encoder hidden state
        h_decoder : decoder hidden state
        '''
        wh_encoder = self.w_encoder(h_encoder)
        wh_decoder = self.w_decoder(tf.expand_dims(h_decoder, 1))

        print('[h_encoder] shape :', h_encoder.shape)
        print('[w_encoder x h_encoder] shape :', wh_encoder.shape)
        
        print('[h_decoder] shape :', h_decoder.shape)
        print('[w_decoder x h_decoder] shape :', wh_decoder.shape)

        # attention score
        at_score = self.w_combine(tf.nn.tanh(wh_decoder + wh_encoder))
        print('[score_alignment] shape :', at_score.shape)

        # attention distribution(weight)
        at_weight = tf.nn.softmax(at_score, axis=1)
        print('\n최종 attention weight :', at_weight.numpy())

        # attention value?(convext vector 얻기)
        context_v = at_weight * wh_decoder
        context_v = tf.reduce_sum(context_vector, axis=1)

        return context_v, at_weight


In [ ]:
w_size = 100

print(f'Hidden State를 {w_size}차원으로 Mapping\n')

### 아래 미완성
###############

# Luong Attention

## 이론

## 코드 구현

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()
        self.W_combine = tf.keras.layers.Dense(units)

    def call(self, )
    ### 아래 미완성
    ###############

# 양방향 LSTM과 어텐션 메커니즘 (IMDB 리뷰데이터)